In [ ]:
from glob import glob
from os import path
import xml
import pandas as pd
FOLDER_NAME = "VOCdevkit\\VOC2012\\JPEGImages\\"

In [ ]:
import tensorflow

In [ ]:
def find_file(folder):
    return glob(path.join(folder,"*.{}".format('xml')))

def text_format(x_min, y_min, x_max, y_max, part_name,file_path, folder_name, img_width, img_height, img_depth):
    x_min = str(int(float(x_min)))
    y_min = str(int(float(y_min)))
    x_max = str(int(float(x_max)))
    y_max = str(int(float(y_max)))
    img_width = str(int(float(img_width)))
    img_height = str(int(float(img_height)))
    img_depth = str(int(float(img_depth)))
    return "{0}, {1}, {2}, {3}, {4}, {5}, {6}, {7}, {8}, {9}\n". format(x_min, y_min, x_max, y_max, part_name,FOLDER_NAME+file_path, folder_name, img_width, img_height, img_depth)

def get_db_desc(annotation_file):
    xml_tree = ET.parse(annotation_file)
    root = xml_tree.getroot()
    folder_name = root.findall('folder')[0].text
    file_path = root.findall('filename')[0].text
    sizes =  root.findall('size')[0]
    img_width = sizes.findall('width')[0].text
    img_height = sizes.findall('height')[0].text
    img_depth = sizes.findall('depth')[0].text
    objects = root.findall('object')
    desc = ""
    for obj in objects:
        part_name = obj.findall('name')[0].text
        bndbox = obj.findall('bndbox')[0]
        x_min = bndbox.findall('xmin')[0].text
        y_min = bndbox.findall('ymin')[0].text
        x_max = bndbox.findall('xmax')[0].text
        y_max = bndbox.findall('ymax')[0].text
        desc +=text_format(x_min, y_min, x_max, y_max, part_name, file_path, folder_name, img_width, img_height, img_depth)
        parts = obj.findall('part')
        for part in parts:
            part_name = part.findall('name')[0].text
            bndbox = part.findall('bndbox')[0]
            x_min = bndbox.findall('xmin')[0].text
            y_min = bndbox.findall('ymin')[0].text
            x_max = bndbox.findall('xmax')[0].text
            y_max = bndbox.findall('ymax')[0].text
            desc +=text_format(x_min, y_min, x_max, y_max, part_name,file_name, folder_name, img_width, img_height, img_depth)
    return desc

In [ ]:
def get_full_db_desc():
    annotation_files = find_file('Annotations')
    desc = "xmin,ymin,xmax,ymax,part_name,file_path,folder_name,image_width,image_height,image_depth\n"
    for annotation_file in annotation_files:
        desc +=get_db_desc(annotation_file)
    return desc

In [ ]:
desc = get_full_db_desc()

In [ ]:
def generate_csv(desc):
    f = open("data.csv", 'w')
    f.write(desc)
    f.close()

In [ ]:
generate_csv(desc)

In [ ]:
dataset = pd.read_csv("data.csv")

In [ ]:
training = dataset.sample(n=int(0.7*len(dataset)))

In [ ]:
training = dataset.sample(n=int(0.7*len(dataset)))

In [ ]:
training = training[['file_path','xmin', 'ymin', 'xmax', 'ymax', 'part_name']]

In [ ]:
training

In [ ]:
training.to_csv("training.csv", header=False, index=False)

In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [ ]:
torch.cuda.current_device()

In [3]:
from keras import backend as K

In [5]:
from keras.layers import Layer
l = Layer()

In [23]:
running_mean = l.add_weight(shape=(256,), initializer='zero',
                                            name='{}_running_mean'.format("jordan"),
                                            trainable=False)

In [26]:
broadcast_shape = (16, 4, 1, 4)

In [27]:
broadcast_running_mean = K.reshape(running_mean, broadcast_shape)

In [18]:
help(K.reshape)

Help on function reshape in module keras.backend:

reshape(x, shape)
    Reshapes a tensor to the specified shape.
    
    Args:
        x: Tensor or variable.
        shape: Target shape tuple.
    
    Returns:
        A tensor.
    
    Example:
    
      >>> a = tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])
      >>> a
      <tf.Tensor: shape=(4, 3), dtype=int32, numpy=
      array([[ 1,  2,  3],
             [ 4,  5,  6],
             [ 7,  8,  9],
             [10, 11, 12]], dtype=int32)>
      >>> tf.keras.backend.reshape(a, shape=(2, 6))
      <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
      array([[ 1,  2,  3,  4,  5,  6],
             [ 7,  8,  9, 10, 11, 12]], dtype=int32)>



In [28]:
from keras import initializers, regularizers

In [29]:
gamma_init = initializers.get('one')

In [37]:
help(regularizers.get)

Help on function get in module keras.regularizers:

get(identifier)
    Retrieve a regularizer instance from a config or identifier.



In [35]:
beta_regularizer = regularizers.get(None)
gamma_regularizer = regularizers.get(None)

In [41]:
shape = (None, )
gamma = l.add_weight(initializer=gamma_init,
                                     regularizer=gamma_regularizer,
                                     name='{}_gamma'.format('test'),
                                     trainable=False)

In [40]:
help(l.add_weight)

Help on method add_weight in module keras.engine.base_layer:

add_weight(name=None, shape=None, dtype=None, initializer=None, regularizer=None, trainable=None, constraint=None, use_resource=None, synchronization=<VariableSynchronization.AUTO: 0>, aggregation=<VariableAggregationV2.NONE: 0>, **kwargs) method of keras.engine.base_layer.Layer instance
    Adds a new variable to the layer.
    
    Args:
      name: Variable name.
      shape: Variable shape. Defaults to scalar if unspecified.
      dtype: The type of the variable. Defaults to `self.dtype`.
      initializer: Initializer instance (callable).
      regularizer: Regularizer instance (callable).
      trainable: Boolean, whether the variable should be part of the layer's
        "trainable_variables" (e.g. variables, biases)
        or "non_trainable_variables" (e.g. BatchNorm mean and variance).
        Note that `trainable` cannot be `True` if `synchronization`
        is set to `ON_READ`.
      constraint: Constraint insta